In [255]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from sklearn import metrics
import catboost as cb
from sklearn.ensemble import RandomForestRegressor

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error

from sklearn.model_selection import train_test_split

from sklearn.impute import SimpleImputer

from mlxtend.preprocessing import minmax_scaling
# for Box-Cox Transformation
from scipy import stats
import xgboost as xgb
from sklearn.metrics import roc_auc_score

from sklearn import preprocessing
import lightgbm as lgbm
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier



from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline


from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
import warnings 
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
warnings.filterwarnings('ignore')

In [6]:
train = pd.read_csv('/kaggle/input/alidata/final_train.csv')
test = pd.read_csv('/kaggle/input/alidata/final_test.csv')
submition = pd.read_csv('/kaggle/input/financialresilience1234/SampleSubmission.csv')

In [7]:
df = pd.read_csv('/kaggle/input/financialresilience1234/Train.csv')
y = df.target

In [8]:
train.drop(["Q2"],axis=1,inplace=True)
train.drop(["Q3"],axis=1,inplace=True)
train.drop(["Q4"],axis=1,inplace=True)
train.drop(["Q5"],axis=1,inplace=True)
train.drop(["Q8"],axis=1,inplace=True)
train.drop(["Q9"],axis=1,inplace=True)
train.drop(["Q20"],axis=1,inplace=True)
train.drop(["Q21"],axis=1,inplace=True)
train.drop(["Q23"],axis=1,inplace=True)
train.drop(["Q28"],axis=1,inplace=True)
test.drop(["Q2"],axis=1,inplace=True)
test.drop(["Q3"],axis=1,inplace=True)
test.drop(["Q4"],axis=1,inplace=True)
test.drop(["Q5"],axis=1,inplace=True)
test.drop(["Q8"],axis=1,inplace=True)
test.drop(["Q9"],axis=1,inplace=True)
test.drop(["Q20"],axis=1,inplace=True)
test.drop(["Q21"],axis=1,inplace=True)
test.drop(["Q23"],axis=1,inplace=True)
test.drop(["Q28"],axis=1,inplace=True)
train.drop(["owns_mobile"], axis=1, inplace=True)

ntrain = train.shape[0] # to be used to split train and test set from the combined dataframe

all_data = pd.concat((train, test)).reset_index(drop=True)

all_data["debit_card"]=all_data["Q1"]+all_data["Q6"]
all_data["got_savings"]=all_data["Q11"]+all_data["Q12"]+all_data["Q13a"]+all_data["Q22"]
all_data["borrowed"]=all_data["Q16"]+all_data["Q14"]+all_data["Q15"]+all_data["Q17a"]+all_data["Q17b"]
all_data["given"]=all_data["Q19"]
all_data["benefit"]=all_data["Q24"]+all_data["Q25"]+all_data["Q26"]

all_data["internet_access"]=all_data["Q10a"]+all_data["Q10b"]
all_data.drop(["Q10b","Q10a","Q27"], axis=1, inplace=True)

all_data.drop(["Q6","Q11","Q12","Q13a","Q22","Q16","Q14","Q15","Q17a","Q17b","Q19","Q24","Q25","Q26","Q1"], axis=1, inplace=True)

all_data.drop(["owns_mobile"], axis=1, inplace=True)

#all_data['ID']=all_IDs["ID"]
#all_data.drop(["internet_access"], axis=1, inplace=True)
all_data.drop(["region"], axis=1, inplace=True)


train = all_data[:ntrain]
test = all_data[ntrain:]

In [9]:
#train data scaling
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

In [11]:
scaled_train_data=scaler.fit_transform(train, y=y)
scaled_test_data=scaler.transform(test)

In [26]:
# seed = 7
# test_size = 0.33
# X_train, X_test, y_train, y_test = train_test_split(scaled_train_data, y, test_size=test_size, random_state=seed)

seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(train, y, test_size=test_size, random_state=seed)

In [27]:
from sklearn.naive_bayes import GaussianNB
# X, y = load_iris(return_X_y=True)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)
gnb = GaussianNB()

gnb.fit(X_train, y_train)
gnb.score(X_test,y_test)

0.6451295980581964

In [28]:
lr = LinearRegression()

In [29]:
lr.fit(X_train,y_train)

LinearRegression()

In [30]:
lr.score(X_test,y_test)

0.2000077907300779

In [38]:
gnb.predict(test).max()

0

In [48]:
from sklearn.model_selection import KFold, StratifiedKFold, RepeatedStratifiedKFold
from sklearn.linear_model import LassoCV

In [51]:
preds = []
scores = []

skf = StratifiedKFold(n_splits=20, random_state=42, shuffle=True)

model_params = {
    'precompute': 'auto',
    'fit_intercept': True,
    'max_iter': 100000,
    'verbose': False,
    'eps': 1e-04, 
    'n_alphas': 1000,
    'n_jobs': -1,
}

for train_index, test_index in skf.split(train, y):
    X_train, X_valid = train.iloc[train_index], train.iloc[test_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[test_index]
    
    model = LassoCV(**model_params)
    
    model.fit(X_train, y_train)
    preds.append(model.predict(X_test))
    s = roc_auc_score(y_valid, model.predict(X_valid))
    print(f'Best val auc: {s: .4f}')
    scores.append(s)
    
print(f'mean scores: {np.mean(scores): .4f}')

Best val auc:  0.7689
Best val auc:  0.7646
Best val auc:  0.7686
Best val auc:  0.7646
Best val auc:  0.7547
Best val auc:  0.7737
Best val auc:  0.7679
Best val auc:  0.7591
Best val auc:  0.7567
Best val auc:  0.7592
Best val auc:  0.7714
Best val auc:  0.7707
Best val auc:  0.7639
Best val auc:  0.7644
Best val auc:  0.7763
Best val auc:  0.7729
Best val auc:  0.7521
Best val auc:  0.7735
Best val auc:  0.7594
Best val auc:  0.7666
mean scores:  0.7655


# CatBoost

In [52]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [ ]:
print('ali')

In [57]:
df = pd.read_csv('/kaggle/input/alidata/final_train.csv')
td = pd.read_csv('/kaggle/input/alidata/final_test.csv')

In [59]:
(len(df))==len(train)

True

In [97]:
train.columns,df.columns

(Index(['country', 'population', 'age', 'debit_card', 'got_savings', 'borrowed',
        'given', 'benefit', 'internet_access'],
       dtype='object'),
 Index(['country', 'region', 'population', 'age', 'Q1', 'Q2', 'Q3', 'Q4', 'Q5',
        'Q6', 'Q8', 'Q9', 'Q10a', 'Q10b', 'Q11', 'Q12', 'Q13a', 'Q14', 'Q15',
        'Q16', 'Q17a', 'Q17b', 'Q19', 'Q20', 'Q21', 'Q22', 'Q23', 'Q24', 'Q25',
        'Q26', 'Q27', 'owns_mobile', 'Q28'],
       dtype='object'))

In [98]:
q = ['Q1', 'Q2', 'Q3', 'Q4', 'Q5',
        'Q6', 'Q8', 'Q9', 'Q10a', 'Q10b', 'Q11', 'Q12', 'Q13a', 'Q14', 'Q15',
        'Q16', 'Q17a', 'Q17b', 'Q19', 'Q20', 'Q21', 'Q22', 'Q23', 'Q24', 'Q25',
        'Q26', 'Q27']

In [129]:
td[q]

,Q1,Q2,Q3,Q4,Q5,Q6,Q8,Q9,Q10a,Q10b,...,Q17b,Q19,Q20,Q21,Q22,Q23,Q24,Q25,Q26,Q27
0,0.463494,0.462482,0.468711,0.635905,0.627106,0.815513,0.749355,0.741729,0.521591,0.372149,...,0.596714,0.485135,0.511531,0.476220,0.714582,0.783209,0.579284,0.574065,0.522594,0.684246
1,0.463494,0.462482,0.468711,0.435747,0.435747,0.526127,0.428313,0.428313,0.521591,0.521672,...,0.596714,0.485135,0.511531,0.476220,0.508177,0.507631,0.579284,0.574065,0.522594,0.470778
2,0.724670,0.730990,0.773343,0.635905,0.627106,0.526127,0.518637,0.531898,0.521591,0.521672,...,0.596714,0.485135,0.511531,0.476220,0.508177,0.507631,0.579284,0.643645,0.522594,0.655313
3,0.724670,0.730990,0.773343,0.809158,0.627106,0.815513,0.749355,0.741729,0.800941,0.825926,...,0.596714,0.747652,0.747652,0.660487,0.714582,0.691060,0.604052,0.643645,0.747652,0.684246
4,0.724670,0.730990,0.773343,0.635905,0.627106,0.526127,0.518637,0.741729,0.521591,0.521672,...,0.535388,0.485135,0.601296,0.476220,0.508177,0.507631,0.579284,0.574065,0.522594,0.470778
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44938,0.724670,0.730990,0.773343,0.635905,0.795387,0.815513,0.749355,0.741729,0.521591,0.521672,...,0.596714,0.485135,0.511531,0.660487,0.508177,0.507631,0.579284,0.574065,0.522594,0.470778
44939,0.724670,0.730990,0.629663,0.635905,0.627106,0.526127,0.518637,0.531898,0.521591,0.521672,...,0.596714,0.485135,0.601296,0.660487,0.714582,0.691060,0.579284,0.574065,0.522594,0.684246
44940,0.463494,0.462482,0.468711,0.435747,0.435747,0.526127,0.428313,0.428313,0.521591,0.521672,...,0.535388,0.485135,0.601296,0.476220,0.508177,0.507631,0.579284,0.574065,0.522594,0.655313
44941,0.463494,0.462482,0.468711,0.435747,0.435747,0.526127,0.428313,0.428313,0.521591,0.521672,...,0.535388,0.718690,0.511531,0.660487,0.508177,0.507631,0.579284,0.574065,0.522594,0.470778


In [100]:
full_concat = pd.concat([df[q],train],axis=1)
test_concat = pd.concat([td[q] ,test],axis=1)

In [130]:
test_concat = pd.concat([td[q],test],axis=1)

In [162]:
test_concat = pd.concat([td[q].reset_index(drop=True), test.reset_index(drop=True)], axis=1)

In [103]:
seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(full_concat, y, test_size=test_size, random_state=seed)

In [76]:
preds = []
scores = []

skf = StratifiedKFold(n_splits=20, random_state=42, shuffle=True)

model_params = {
    'precompute': 'auto',
    'fit_intercept': True,
    'max_iter': 100000,
    'verbose': False,
    'eps': 1e-04, 
    'n_alphas': 1000,
    'n_jobs': -1,
}

for train_index, test_index in skf.split(full_concat, y):
    X_train, X_valid = full_concat.iloc[train_index], full_concat.iloc[test_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[test_index]
    
    model = LassoCV(**model_params)
    
    model.fit(X_train, y_train)
    preds.append(model.predict(X_test))
    s = roc_auc_score(y_valid, model.predict(X_valid))
    print(f'Best val auc: {s: .4f}')
    scores.append(s)
    
print(f'mean scores: {np.mean(scores): .4f}')

Best val auc:  0.7892
Best val auc:  0.7810
Best val auc:  0.7808
Best val auc:  0.7795
Best val auc:  0.7713
Best val auc:  0.7899
Best val auc:  0.7805
Best val auc:  0.7756
Best val auc:  0.7661
Best val auc:  0.7756
Best val auc:  0.7850
Best val auc:  0.7865
Best val auc:  0.7809
Best val auc:  0.7862
Best val auc:  0.7926
Best val auc:  0.7911
Best val auc:  0.7672
Best val auc:  0.7835
Best val auc:  0.7729
Best val auc:  0.7847
mean scores:  0.7810


In [78]:
from scipy.stats import rankdata

In [79]:
rankpreds = []

for pred in preds:
    rankpred = rankdata(pred)
    rankpreds.append(rankpred)

finalrank = np.average(np.array(rankpreds), axis=0) / len(X_test)

In [82]:
# submition['targeg'] = finalrank

,country,population,age,debit_card,got_savings,borrowed,given,benefit,internet_access
0,0.478657,2.836511,66.0,0.989621,2.079791,2.696864,0.485135,1.700711,1.043263
1,0.829662,2.730439,44.0,0.989621,2.408512,3.029042,0.485135,1.675943,1.043263
2,0.478657,2.836511,29.0,1.250797,2.286197,2.890727,0.485135,1.675943,1.043263
3,0.871612,2.846823,44.0,1.250797,2.906658,3.029042,0.747652,1.925770,1.626868
4,0.216374,2.909528,36.0,0.989621,2.079791,2.890727,0.485135,1.700711,1.322613
...,...,...,...,...,...,...,...,...,...
104862,0.452849,3.028820,48.0,0.989621,2.079791,2.890727,0.485135,1.675943,1.043263
104863,0.753890,2.753077,16.0,1.250797,2.614918,2.890727,0.747652,1.925770,1.626868
104864,0.455587,2.627547,53.0,1.250797,2.079791,2.890727,0.747652,1.901002,1.043263
104865,0.889855,2.895362,50.0,0.989621,2.573029,2.890727,0.747652,1.901002,1.043263


In [105]:
# Created the Xgboost specific DMatrix data format from the numpy array to optimise memory consumption
dtrain = xgb.DMatrix(X_train, y_train)
dvalid = xgb.DMatrix(X_test, label=y)

In [113]:
def objective(trial):
    
# params specifies the XGBoost hyperparameters to be tuned
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 400, 600),
        'max_depth': trial.suggest_int('max_depth', 10, 20),
        'learning_rate': trial.suggest_uniform('learning_rate', 0.01, .1),
        'subsample': trial.suggest_uniform('subsample', 0.50, 1),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.50, 1),
        'gamma': trial.suggest_int('gamma', 0, 10),
        'tree_method': 'gpu_hist',  
        'objective': 'binary:logistic'
    }
    
    bst = xgb.train(params, dtrain)
    preds = bst.predict(dvalid)
    pred_labels = np.rint(preds)
# trials will be evaluated based on their accuracy on the test set
    accuracy = roc_auc_score(y_test, pred_labels)
    return accuracy

In [114]:
import optuna

In [115]:
study = optuna.create_study()
study.optimize(objective,n_trials=5) 
#You can increase n_trials parameter

[I 2023-03-11 06:50:23,527] A new study created in memory with name: no-name-0f4cef97-7dd9-4868-8be3-2f2d8251ccb3


[06:50:23] WARNING: ../src/learner.cc:627: 
Parameters: { "n_estimators" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




[I 2023-03-11 06:50:24,606] Trial 0 finished with value: 0.7064224818916403 and parameters: {'n_estimators': 458, 'max_depth': 14, 'learning_rate': 0.03408503403182764, 'subsample': 0.757275945226354, 'colsample_bytree': 0.8940870774446547, 'gamma': 2}. Best is trial 0 with value: 0.7064224818916403.


[06:50:24] WARNING: ../src/learner.cc:627: 
Parameters: { "n_estimators" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




[I 2023-03-11 06:50:25,010] Trial 1 finished with value: 0.7080083083701739 and parameters: {'n_estimators': 520, 'max_depth': 14, 'learning_rate': 0.09821063058816826, 'subsample': 0.5681121353767837, 'colsample_bytree': 0.6216767820397509, 'gamma': 3}. Best is trial 0 with value: 0.7064224818916403.


[06:50:25] WARNING: ../src/learner.cc:627: 
Parameters: { "n_estimators" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




[I 2023-03-11 06:50:25,476] Trial 2 finished with value: 0.7061367192735473 and parameters: {'n_estimators': 503, 'max_depth': 17, 'learning_rate': 0.06844339938380291, 'subsample': 0.9627237658908697, 'colsample_bytree': 0.6823673314982888, 'gamma': 4}. Best is trial 2 with value: 0.7061367192735473.
[I 2023-03-11 06:50:25,680] Trial 3 finished with value: 0.7015786263252413 and parameters: {'n_estimators': 413, 'max_depth': 17, 'learning_rate': 0.03461306969432192, 'subsample': 0.5799246570139402, 'colsample_bytree': 0.6102758000236177, 'gamma': 6}. Best is trial 3 with value: 0.7015786263252413.


[06:50:25] WARNING: ../src/learner.cc:627: 
Parameters: { "n_estimators" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[06:50:25] WARNING: ../src/learner.cc:627: 
Parameters: { "n_estimators" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




[I 2023-03-11 06:50:25,981] Trial 4 finished with value: 0.7024952498869674 and parameters: {'n_estimators': 567, 'max_depth': 15, 'learning_rate': 0.02869155301555295, 'subsample': 0.9411788816549234, 'colsample_bytree': 0.6877206357728521, 'gamma': 6}. Best is trial 3 with value: 0.7015786263252413.


In [116]:
print('Best trial: score {}, params {}'.format(study.best_trial.value, study.best_trial.params))

Best trial: score 0.7015786263252413, params {'n_estimators': 413, 'max_depth': 17, 'learning_rate': 0.03461306969432192, 'subsample': 0.5799246570139402, 'colsample_bytree': 0.6102758000236177, 'gamma': 6}


In [118]:
best_params = study.best_trial.params
best_params['tree_method'] = 'gpu_hist'      #gpu_hist is really fast
best_params['objective'] = 'binary:logistic'

In [119]:
# Fit the XGBoost classifier with optimal hyperparameters
clf = xgb.XGBClassifier(**best_params)

In [120]:
%time clf.fit(full_concat, y)  #Used the whole training data

CPU times: user 2.48 s, sys: 38.6 ms, total: 2.52 s
Wall time: 2.36 s


XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1,
              colsample_bytree=0.6102758000236177, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, gamma=6, gpu_id=0,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.03461306969432192,
              max_bin=256, max_cat_to_onehot=4, max_delta_step=0, max_depth=17,
              max_leaves=0, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=413, n_jobs=0,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [163]:
clfpred = clf.predict(test_concat)

In [164]:
submition['target'] = clfpred

In [165]:
(len(test_concat))

44943

In [170]:
submition.to_csv('sub:9AM.csv',index=False)

In [169]:
clf.predict_proba(test_concat)

array([[0.18053663, 0.8194634 ],
       [0.74901533, 0.2509847 ],
       [0.5991913 , 0.40080866],
       ...,
       [0.31369203, 0.68630797],
       [0.7704941 , 0.22950591],
       [0.7217182 , 0.2782818 ]], dtype=float32)

In [172]:
clf.predict_proba(test_concat)[:,0]

array([0.18053663, 0.74901533, 0.5991913 , ..., 0.31369203, 0.7704941 ,
       0.7217182 ], dtype=float32)

In [174]:
submition['target'] = clf.predict_proba(test_concat)[:,1]

In [175]:
submition.to_csv('sub:9Am.csv',index=False)

In [176]:
submition['target'] = clf.predict_proba(test_concat)[:,0]
submition.to_csv('sub:9Am2.csv',index=False)

In [178]:
scaled_train_data=scaler.fit_transform(full_concat, y=y)
scaled_test_data=scaler.transform(test_concat)

# Lasso

In [181]:
seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(scaled_train_data, y, test_size=test_size, random_state=seed)
preds = []
scores = []

skf = StratifiedKFold(n_splits=20, random_state=42, shuffle=True)

model_params = {
    'precompute': 'auto',
    'fit_intercept': True,
    'max_iter': 100000,
    'verbose': False,
    'eps': 1e-04, 
    'n_alphas': 1000,
    'n_jobs': -1,
}

for train_index, test_index in skf.split(full_concat, y):
    X_train, X_valid = full_concat.iloc[train_index], full_concat.iloc[test_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[test_index]
    
    model = LassoCV(**model_params)
    
    model.fit(X_train, y_train)
    preds.append(model.predict(scaled_test_data))
    s = roc_auc_score(y_valid, model.predict(X_valid))
    print(f'Best val auc: {s: .4f}')
    scores.append(s)
    
print(f'mean scores: {np.mean(scores): .4f}')

Best val auc:  0.7849
Best val auc:  0.7757
Best val auc:  0.7782
Best val auc:  0.7764
Best val auc:  0.7693
Best val auc:  0.7868
Best val auc:  0.7771
Best val auc:  0.7717
Best val auc:  0.7635
Best val auc:  0.7716
Best val auc:  0.7794
Best val auc:  0.7838
Best val auc:  0.7771
Best val auc:  0.7833
Best val auc:  0.7876
Best val auc:  0.7859
Best val auc:  0.7642
Best val auc:  0.7793
Best val auc:  0.7689
Best val auc:  0.7802
mean scores:  0.7772


In [194]:
finalrank

array([0.2244621 , 0.54297332, 0.25117482, ..., 0.29967626, 0.57514741,
       0.95313731])

In [193]:
rankpreds = []

for pred in preds:
    rankpred = rankdata(pred)
    rankpreds.append(rankpred)

finalrank = np.average(np.array(rankpreds), axis=0) / len(scaled_test_data)

In [195]:
submition['target'] = finalrank
submition.to_csv('sub:9:30Am.csv',index=False)

In [201]:
X_train, X_test, y_train, y_test = train_test_split(scaled_train_data, y, test_size=test_size, random_state=seed)

In [203]:
# Created the Xgboost specific DMatrix data format from the numpy array to optimise memory consumption
dtrain = xgb.DMatrix(X_train, y_train)
dvalid = xgb.DMatrix(X_test, label=y_test)

def objective(trial):
    
# params specifies the XGBoost hyperparameters to be tuned
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 400, 600),
        'max_depth': trial.suggest_int('max_depth', 10, 20),
        'learning_rate': trial.suggest_uniform('learning_rate', 0.01, .1),
        'subsample': trial.suggest_uniform('subsample', 0.50, 1),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.50, 1),
        'gamma': trial.suggest_int('gamma', 0, 10),
        'tree_method': 'gpu_hist',  
        'objective': 'binary:logistic'
    }
    
    bst = xgb.train(params, dtrain)
    preds = bst.predict(dvalid)
    pred_labels = np.rint(preds)
# trials will be evaluated based on their accuracy on the test set
    accuracy = roc_auc_score(y_test, pred_labels)
    return accuracy

In [204]:
study = optuna.create_study()
study.optimize(objective,n_trials=5) 
#You can increase n_trials parameter

[I 2023-03-11 07:40:17,721] A new study created in memory with name: no-name-88d82ef8-4445-4a5e-bb90-a595722d2f08


[07:40:17] WARNING: ../src/learner.cc:627: 
Parameters: { "n_estimators" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




[I 2023-03-11 07:40:18,234] Trial 0 finished with value: 0.7021419333431369 and parameters: {'n_estimators': 430, 'max_depth': 12, 'learning_rate': 0.06601862746773267, 'subsample': 0.7439172605211669, 'colsample_bytree': 0.9648586592472921, 'gamma': 5}. Best is trial 0 with value: 0.7021419333431369.


[07:40:18] WARNING: ../src/learner.cc:627: 
Parameters: { "n_estimators" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




[I 2023-03-11 07:40:19,098] Trial 1 finished with value: 0.7032601440579112 and parameters: {'n_estimators': 585, 'max_depth': 13, 'learning_rate': 0.03775198260642926, 'subsample': 0.5303661887992503, 'colsample_bytree': 0.7186025315595326, 'gamma': 1}. Best is trial 0 with value: 0.7021419333431369.


[07:40:19] WARNING: ../src/learner.cc:627: 
Parameters: { "n_estimators" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




[I 2023-03-11 07:40:19,322] Trial 2 finished with value: 0.7039562691379937 and parameters: {'n_estimators': 449, 'max_depth': 13, 'learning_rate': 0.08340263326501436, 'subsample': 0.669951942899752, 'colsample_bytree': 0.9713706256699897, 'gamma': 6}. Best is trial 0 with value: 0.7021419333431369.
[I 2023-03-11 07:40:19,471] Trial 3 finished with value: 0.7016484254784991 and parameters: {'n_estimators': 501, 'max_depth': 15, 'learning_rate': 0.09024743627244164, 'subsample': 0.8621185909886075, 'colsample_bytree': 0.7762445628567441, 'gamma': 9}. Best is trial 3 with value: 0.7016484254784991.


[07:40:19] WARNING: ../src/learner.cc:627: 
Parameters: { "n_estimators" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[07:40:19] WARNING: ../src/learner.cc:627: 
Parameters: { "n_estimators" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




[I 2023-03-11 07:40:19,575] Trial 4 finished with value: 0.6968910797282006 and parameters: {'n_estimators': 596, 'max_depth': 10, 'learning_rate': 0.058783023024490504, 'subsample': 0.700108319733413, 'colsample_bytree': 0.580930847786134, 'gamma': 10}. Best is trial 4 with value: 0.6968910797282006.


In [205]:
print('Best trial: score {}, params {}'.format(study.best_trial.value, study.best_trial.params))
best_params = study.best_trial.params
best_params['tree_method'] = 'gpu_hist'      #gpu_hist is really fast
best_params['objective'] = 'binary:logistic'


Best trial: score 0.6968910797282006, params {'n_estimators': 596, 'max_depth': 10, 'learning_rate': 0.058783023024490504, 'subsample': 0.700108319733413, 'colsample_bytree': 0.580930847786134, 'gamma': 10}


In [206]:
# Fit the XGBoost classifier with optimal hyperparameters
clf = xgb.XGBClassifier(**best_params)

In [207]:
%time clf.fit(full_concat, y)  #Used the whole training data

CPU times: user 1.24 s, sys: 26.8 ms, total: 1.27 s
Wall time: 1.12 s


XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1,
              colsample_bytree=0.580930847786134, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, gamma=10, gpu_id=0,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.058783023024490504,
              max_bin=256, max_cat_to_onehot=4, max_delta_step=0, max_depth=10,
              max_leaves=0, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=596, n_jobs=0,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [212]:
clfpred = clf.predict_proba(scaled_test_data)[:,1]
submition['target'] = clfpred
submition.to_csv('sub:9:42AM.csv',index=False)

In [213]:
clfpred = clf.predict(scaled_test_data)
submition['target'] = clfpred
submition.to_csv('sub:9:45AM.csv',index=False)

In [ ]:
kf = StratifiedKFold(n_splits=10, random_state=42, shuffle=True)

cb_params = {
    'depth': 3,
    'learning_rate': 0.01,
    'rsm': 0.5,
    'subsample': 0.931,
    'l2_leaf_reg': 69,
    'min_data_in_leaf': 20,
    'random_strength': 0.175,
    'use_best_model': True,
    'task_type': 'CPU',
    'bootstrap_type': 'Bernoulli',
    'grow_policy': 'SymmetricTree',
    'loss_function': 'Logloss',
    'eval_metric': 'AUC',
    'scale_pos_weight': 5
}

for train_index, val_index in skf.split(X, y):
    cb_train = cb.Pool(data=X.iloc[train_index], label=y.iloc[train_index])
    cb_valid = cb.Pool(data=X.iloc[val_index], label=y.iloc[val_index])
    
    model = cb.train(
        params=cb_params,
        dtrain=cb_train,
        num_boost_round=10000,
        evals=cb_valid, 
        early_stopping_rounds=500,
        verbose=False
    )
    
    s = roc_auc_score(y[val_index], model.predict(cb_valid))
    
    print(f"Best val auc: {s:.4f}")
    scores.append(s)
    preds.append(sigmoid(model.predict(X_test)))

print(f'mean scores: {np.mean(scores): .4f}')

# Feature Engigeering


In [218]:
df = pd.read_csv('/kaggle/input/financialresilience1234/Train.csv')
var= pd.read_csv('/kaggle/input/financialresilience1234/VariableDefinitions.csv')

In [219]:
var

,ID,Person unique/random identification,"For columns(Q1 - Q28), Value 1 - Yes, 2 - No, 3-Don’t Know 4 - refused to answer"
0,country,The country,NaN
1,region,"Region of a given country,.i.e., South Asia, E...",NaN
2,population,The population of the given country,NaN
3,age,Individual age,NaN
4,Q1,Has ATM/debit card,NaN
5,Q2,Is the ATM connect to an account with your name?,NaN
6,Q3,Purchased with the ATM,NaN
7,Q4,Use of mobile or internet to make payments or ...,NaN
8,Q5,Mobile/internet to check account balance,NaN
9,Q6,Has a credit card,NaN


In [234]:
cols_with_missing = [col for col in df.columns
                     if df[col].isnull().any()]

cols_with_missing = ['Q2',
 'Q3',
 'Q4',
 'Q5',
 'Q8',
 'Q9',
 'Q19',
 'Q20',
 'Q23',
 'Q26',
 'Q27']

In [333]:
col_n_q=['country', 'population',
       'age', 'debit_card', 'got_savings', 'borrowed', 'given', 'benefit',
       'internet_access']

In [334]:
full_data = full_concat.drop(cols_with_missing,axis=1)
full_test = test_concat.drop(cols_with_missing,axis=1)
target = y

full_data = full_data[col_n_q]
full_test = full_test[col_n_q]

In [335]:
X = full_data
y = target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [246]:
%%time
parameters = {
    'max_depth': [3, 5, 7, 9], 
    'n_estimators': [100,200,300,400],
    'learning_rate': [0.01, 0.05, 0.1]
}

model_xgb = xgb.XGBClassifier(
    random_state=42,
    tree_method='gpu_hist'
)

model_xgb = GridSearchCV(
    model_xgb, 
    parameters, 
    cv=5,
    scoring='roc_auc'
)

model_xgb.fit(X_train, y_train)

CPU times: user 7min 38s, sys: 2.57 s, total: 7min 41s
Wall time: 7min 6s


GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     gamma=None, gpu_id=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_bin=None,
                                     max_ca...e,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                

In [247]:
cross_valid_scores = {}

In [248]:
print('-----')
print(f'Best parameters {model_xgb.best_params_}')
print(
    f'Mean cross-validated accuracy score of the best_estimator: ' + 
    f'{model_xgb.best_score_:.3f}'
)
cross_valid_scores['xgboost'] = model_xgb.best_score_
print('-----')

-----
Best parameters {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 300}
Mean cross-validated accuracy score of the best_estimator: 0.791
-----


In [ ]:
%%time
parameters = {
    'n_estimators': [5, 10, 15, 20, 25, 50, 100],
    'learning_rate': [0.01, 0.05, 0.1],
    'num_leaves': [7, 15, 31],
}

model_lgbm = lgbm.LGBMClassifier(
    random_state=42,
    device =  "gpu"
)

model_lgbm = GridSearchCV(
    model_lgbm, 
    parameters, 
    cv=5,
    scoring='roc_auc',
)

model_lgbm.fit(
    X_train, 
    y_train
)

print('-----')
print(f'Best parameters {model_lgbm.best_params_}')
print(
    f'Mean cross-validated accuracy score of the best_estimator: ' + 
    f'{model_lgbm.best_score_:.3f}'
)
cross_valid_scores['lightgbm'] = model_lgbm.best_score_
print('-----')

In [280]:
xgb_model = xgb.XGBClassifier(
    random_state=42,
    learning_rate = 0.1,
    max_depth = 5,
    n_estimators = 300,
    tree_method='gpu_hist')

xgb_model.fit(full_data, target)
xgb_model.predict_proba(X_test)[:,1]

array([0.7449452 , 0.87914354, 0.32735634, ..., 0.40478253, 0.47349212,
       0.83124715], dtype=float32)

In [284]:
roc_auc_score(y_test,pred)

0.7871186929600013

In [282]:
submition['target'] = xgb_model.predict_proba(full_test)[:,1]

In [283]:
submition.to_csv('sub10:402AM.csv',index=False)

In [445]:
# Created the Xgboost specific DMatrix data format from the numpy array to optimise memory consumption
dtrain = xgb.DMatrix(X_train, y_train)
dvalid = xgb.DMatrix(X_test, label=y_test)

def objective(trial):
    
# params specifies the XGBoost hyperparameters to be tuned
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 400, 600),
        'max_depth': trial.suggest_int('max_depth', 10, 20),
        'learning_rate': trial.suggest_uniform('learning_rate', 0.01, .1),
        'subsample': trial.suggest_uniform('subsample', 0.50, 1),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.50, 1),
        'gamma': trial.suggest_int('gamma', 0, 10),
        'tree_method': 'gpu_hist',  
        'objective': 'binary:logistic'
    }
    
    bst = xgb.train(params, dtrain)
    preds = bst.predict(dvalid)
    pred_labels = np.rint(preds)
# trials will be evaluated based on their accuracy on the test set
    accuracy = roc_auc_score(y_test, pred_labels)
    return accuracy

study = optuna.create_study()
study.optimize(objective,n_trials=5) 
#You can increase n_trials parameter

[I 2023-03-11 09:43:51,060] A new study created in memory with name: no-name-510f04b7-c341-4ad2-a773-d2768bc5b7d9


[09:43:51] WARNING: ../src/learner.cc:627: 
Parameters: { "n_estimators" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




[I 2023-03-11 09:43:52,268] Trial 0 finished with value: 0.6956941249216833 and parameters: {'n_estimators': 530, 'max_depth': 18, 'learning_rate': 0.07244574907073605, 'subsample': 0.6658750269485882, 'colsample_bytree': 0.9633861991674436, 'gamma': 1}. Best is trial 0 with value: 0.6956941249216833.
[I 2023-03-11 09:43:52,401] Trial 1 finished with value: 0.6968366345260121 and parameters: {'n_estimators': 510, 'max_depth': 14, 'learning_rate': 0.08134490138656023, 'subsample': 0.967941281944309, 'colsample_bytree': 0.8436012773186158, 'gamma': 6}. Best is trial 0 with value: 0.6956941249216833.


[09:43:52] WARNING: ../src/learner.cc:627: 
Parameters: { "n_estimators" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[09:43:52] WARNING: ../src/learner.cc:627: 
Parameters: { "n_estimators" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




[I 2023-03-11 09:43:52,533] Trial 2 finished with value: 0.6933279980099776 and parameters: {'n_estimators': 525, 'max_depth': 15, 'learning_rate': 0.09946550353620924, 'subsample': 0.5367533979077161, 'colsample_bytree': 0.7288340783773415, 'gamma': 4}. Best is trial 2 with value: 0.6933279980099776.
[I 2023-03-11 09:43:52,683] Trial 3 finished with value: 0.6978425003755123 and parameters: {'n_estimators': 547, 'max_depth': 14, 'learning_rate': 0.09861435734890184, 'subsample': 0.8887157367646212, 'colsample_bytree': 0.7734318098734143, 'gamma': 5}. Best is trial 2 with value: 0.6933279980099776.


[09:43:52] WARNING: ../src/learner.cc:627: 
Parameters: { "n_estimators" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[09:43:52] WARNING: ../src/learner.cc:627: 
Parameters: { "n_estimators" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




[I 2023-03-11 09:43:52,852] Trial 4 finished with value: 0.6937059028741689 and parameters: {'n_estimators': 543, 'max_depth': 11, 'learning_rate': 0.06610271399990804, 'subsample': 0.986163416845733, 'colsample_bytree': 0.5032660732743378, 'gamma': 3}. Best is trial 2 with value: 0.6933279980099776.


In [446]:
print('Best trial: score {}, params {}'.format(study.best_trial.value, study.best_trial.params))

Best trial: score 0.6933279980099776, params {'n_estimators': 525, 'max_depth': 15, 'learning_rate': 0.09946550353620924, 'subsample': 0.5367533979077161, 'colsample_bytree': 0.7288340783773415, 'gamma': 4}


In [447]:
best_params = study.best_trial.params
best_params['tree_method'] = 'gpu_hist'      #gpu_hist is really fast
best_params['objective'] = 'binary:logistic'
# Fit the XGBoost classifier with optimal hyperparameters

In [448]:
best_params

{'n_estimators': 525,
 'max_depth': 15,
 'learning_rate': 0.09946550353620924,
 'subsample': 0.5367533979077161,
 'colsample_bytree': 0.7288340783773415,
 'gamma': 4,
 'tree_method': 'gpu_hist',
 'objective': 'binary:logistic'}

In [449]:
clf = xgb.XGBClassifier(**best_params)
%time clf.fit(full_data, y)  #Used the whole training data

CPU times: user 1.1 s, sys: 16.6 ms, total: 1.12 s
Wall time: 1.07 s


XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1,
              colsample_bytree=0.7288340783773415, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, gamma=4, gpu_id=0,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.09946550353620924,
              max_bin=256, max_cat_to_onehot=4, max_delta_step=0, max_depth=15,
              max_leaves=0, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=525, n_jobs=0,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [450]:
clfpred = clf.predict_proba(full_test)[:,1]
submition['target'] = clfpred
submition.to_csv('sub:11:4444.csv',index=False)

# Hyperparameter tuning of XGBoost¶


In [427]:
full_data = full_concat.drop(cols_with_missing,axis=1)
full_test = test_concat.drop(cols_with_missing,axis=1)
target = y

full_data = full_data[col_n_q]
full_test = full_test[col_n_q]
full_data.pop('internet_access')
full_test.pop('internet_access')

0        0.893740
1        1.043263
2        1.043263
3        1.626868
4        1.043263
           ...   
44938    1.043263
44939    1.043263
44940    1.043263
44941    1.043263
44942    1.043263
Name: internet_access, Length: 44943, dtype: float64

In [434]:
X = full_data
y = target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [435]:
params={
    "learning_rate":[0.05,0.10,0.15,0.20,0.25,0.30],
    "max_depth":[3,4,5,6,8,10,12,15],
    "min_child_weight":[1,3,5,7],
    "gamma":[0.0,0.1,0.2,0.3,0.4],
    "colsample_bytree":[0.3,0.4,0.5,0.7],
    "n_estimators":[10,20,30,50,100,200,300,400,600,1000]
}

In [436]:
from sklearn.model_selection import RandomizedSearchCV
import xgboost

In [437]:
classifier=xgboost.XGBClassifier(tree_method='gpu_hist',gpu_id=0)

In [438]:
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))

In [439]:
random_search=RandomizedSearchCV(classifier,param_distributions=params,n_iter=5,scoring='roc_auc',cv=5,verbose=3)

In [440]:
from datetime import datetime
# Here we go
start_time = timer(None) # timing starts from this point for "start_time" variable
random_search.fit(X_train,y_train)
timer(start_time) # timing ends here for "start_time" variable

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV 1/5] END colsample_bytree=0.5, gamma=0.1, learning_rate=0.1, max_depth=12, min_child_weight=5, n_estimators=20;, score=0.789 total time=   0.7s
[CV 2/5] END colsample_bytree=0.5, gamma=0.1, learning_rate=0.1, max_depth=12, min_child_weight=5, n_estimators=20;, score=0.787 total time=   0.6s
[CV 3/5] END colsample_bytree=0.5, gamma=0.1, learning_rate=0.1, max_depth=12, min_child_weight=5, n_estimators=20;, score=0.777 total time=   0.6s
[CV 4/5] END colsample_bytree=0.5, gamma=0.1, learning_rate=0.1, max_depth=12, min_child_weight=5, n_estimators=20;, score=0.785 total time=   0.6s
[CV 5/5] END colsample_bytree=0.5, gamma=0.1, learning_rate=0.1, max_depth=12, min_child_weight=5, n_estimators=20;, score=0.787 total time=   0.6s
[CV 1/5] END colsample_bytree=0.7, gamma=0.4, learning_rate=0.05, max_depth=5, min_child_weight=1, n_estimators=100;, score=0.787 total time=   0.2s
[CV 2/5] END colsample_bytree=0.7, gamma=0.4, learn

In [441]:
random_search.best_params_

{'n_estimators': 200,
 'min_child_weight': 1,
 'max_depth': 6,
 'learning_rate': 0.05,
 'gamma': 0.4,
 'colsample_bytree': 0.5}

In [442]:
classifier=xgboost.XGBClassifier(**random_search.best_params_,tree_method='gpu_hist')

In [443]:
classifier.fit(full_data,target)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.5,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0.4, gpu_id=0, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.05, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=200,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [444]:
clfpred = classifier.predict_proba(full_test)[:,1]
submition['target'] = clfpred
submition.to_csv('sub:11:42AM.csv',index=False)

In [319]:
from sklearn.model_selection import cross_val_score
score=cross_val_score(classifier,X_train,y_train,cv=10)

In [320]:
from sklearn.metrics import accuracy_score

In [321]:
accuracy=[]
skf=StratifiedKFold(n_splits=10,random_state=None)
skf.get_n_splits(X,y)
for train_index,test_index in skf.split(X,y):
    X1_train,X1_test=X.iloc[train_index],X.iloc[test_index]
    Y1_train,Y1_test=y.iloc[train_index],y.iloc[test_index]
    
    classifier.fit(X1_train,Y1_train)
    prediction=classifier.predict(X1_test)
    socre=accuracy_score(prediction,Y1_test)
    accuracy.append(score)

In [322]:
print(accuracy)

[array([0.72183626, 0.72537801, 0.7131181 , 0.71911184, 0.71407165,
       0.7169323 , 0.72288828, 0.73106267, 0.71389646, 0.71989101]), array([0.72183626, 0.72537801, 0.7131181 , 0.71911184, 0.71407165,
       0.7169323 , 0.72288828, 0.73106267, 0.71389646, 0.71989101]), array([0.72183626, 0.72537801, 0.7131181 , 0.71911184, 0.71407165,
       0.7169323 , 0.72288828, 0.73106267, 0.71389646, 0.71989101]), array([0.72183626, 0.72537801, 0.7131181 , 0.71911184, 0.71407165,
       0.7169323 , 0.72288828, 0.73106267, 0.71389646, 0.71989101]), array([0.72183626, 0.72537801, 0.7131181 , 0.71911184, 0.71407165,
       0.7169323 , 0.72288828, 0.73106267, 0.71389646, 0.71989101]), array([0.72183626, 0.72537801, 0.7131181 , 0.71911184, 0.71407165,
       0.7169323 , 0.72288828, 0.73106267, 0.71389646, 0.71989101]), array([0.72183626, 0.72537801, 0.7131181 , 0.71911184, 0.71407165,
       0.7169323 , 0.72288828, 0.73106267, 0.71389646, 0.71989101]), array([0.72183626, 0.72537801, 0.7131181 , 0.71

In [329]:
full_data.columns

Index(['Q1', 'Q6', 'Q10a', 'Q10b', 'Q11', 'Q12', 'Q13a', 'Q14', 'Q15', 'Q16',
       'Q17a', 'Q17b', 'Q21', 'Q22', 'Q24', 'Q25', 'country', 'population',
       'age', 'debit_card', 'got_savings', 'borrowed', 'given', 'benefit',
       'internet_access'],
      dtype='object')

In [382]:
full_data.columns

Index(['country', 'population', 'age', 'debit_card', 'got_savings', 'borrowed',
       'given', 'benefit'],
      dtype='object')

,ID,country,region,population,age,Q1,Q2,Q3,Q4,Q5,...,Q21,Q22,Q23,Q24,Q25,Q26,Q27,owns_mobile,Q28,target
0,ID_0000020532,Morocco,Middle East & North Africa (excluding high inc...,2.555017e+07,66.0,2,NaN,NaN,NaN,NaN,...,1,2,NaN,1,2,2.0,2.0,1.0,1.0,0
1,ID_0000075765,Lebanon,Middle East & North Africa (excluding high inc...,4.591023e+06,44.0,2,NaN,NaN,2.0,2.0,...,1,2,NaN,2,2,2.0,1.0,1.0,1.0,1
2,ID_0000085491,Morocco,Middle East & North Africa (excluding high inc...,2.555017e+07,29.0,1,1.0,1.0,2.0,2.0,...,1,1,2.0,2,2,2.0,NaN,1.0,1.0,0
3,ID_0000144505,Canada,High income: OECD,3.049117e+07,44.0,1,1.0,1.0,1.0,1.0,...,1,1,2.0,1,2,NaN,NaN,1.0,NaN,1
4,ID_0000214184,Mexico,Latin America & Caribbean (excluding high income),9.300232e+07,36.0,2,NaN,NaN,NaN,NaN,...,1,2,NaN,1,2,2.0,2.0,1.0,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104862,ID_9999251978,India,South Asia,9.508153e+08,48.0,2,NaN,NaN,NaN,NaN,...,1,2,NaN,2,2,2.0,2.0,2.0,1.0,0
104863,ID_9999305731,"Hong Kong SAR, China",High income: nonOECD,6.522989e+06,16.0,1,1.0,2.0,2.0,2.0,...,2,1,2.0,1,2,NaN,NaN,1.0,NaN,1
104864,ID_9999406850,Mauritius,Sub-Saharan Africa (excluding high income),1.024565e+06,53.0,1,1.0,2.0,2.0,2.0,...,2,2,NaN,2,2,NaN,2.0,2.0,NaN,0
104865,ID_9999627239,Germany,High income: OECD,7.184693e+07,50.0,2,NaN,NaN,NaN,NaN,...,1,2,NaN,2,2,NaN,1.0,1.0,NaN,1


In [419]:
q

['Q1',
 'Q2',
 'Q3',
 'Q4',
 'Q5',
 'Q6',
 'Q8',
 'Q9',
 'Q10a',
 'Q10b',
 'Q11',
 'Q12',
 'Q13a',
 'Q14',
 'Q15',
 'Q16',
 'Q17a',
 'Q17b',
 'Q19',
 'Q20',
 'Q21',
 'Q22',
 'Q23',
 'Q24',
 'Q25',
 'Q26',
 'Q27',
 'target']

In [428]:
dr = df[q]
dr

,Q1,Q2,Q3,Q4,Q5,Q6,Q8,Q9,Q10a,Q10b,...,Q19,Q20,Q21,Q22,Q23,Q24,Q25,Q26,Q27,target
0,2,NaN,NaN,NaN,NaN,2,NaN,NaN,2,2,...,2.0,1.0,1,2,NaN,1,2,2.0,2.0,0
1,2,NaN,NaN,2.0,2.0,2,1.0,1.0,2,2,...,2.0,2.0,1,2,NaN,2,2,2.0,1.0,1
2,1,1.0,1.0,2.0,2.0,2,1.0,1.0,2,2,...,2.0,2.0,1,1,2.0,2,2,2.0,NaN,0
3,1,1.0,1.0,1.0,1.0,2,1.0,1.0,1,1,...,NaN,NaN,1,1,2.0,1,2,NaN,NaN,1
4,2,NaN,NaN,NaN,NaN,2,NaN,NaN,1,2,...,2.0,2.0,1,2,NaN,1,2,2.0,2.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104862,2,NaN,NaN,NaN,NaN,2,NaN,NaN,2,2,...,2.0,2.0,1,2,NaN,2,2,2.0,2.0,0
104863,1,1.0,2.0,2.0,2.0,2,1.0,1.0,1,1,...,NaN,NaN,2,1,2.0,1,2,NaN,NaN,1
104864,1,1.0,2.0,2.0,2.0,2,2.0,1.0,2,2,...,NaN,NaN,2,2,NaN,2,2,NaN,2.0,0
104865,2,NaN,NaN,NaN,NaN,2,NaN,NaN,2,2,...,NaN,NaN,1,2,NaN,2,2,NaN,1.0,1


In [429]:
dt = pd.concat([full_data.reset_index(drop=True), dr.reset_index(drop=True)], axis=1)

In [426]:
dt

,country,country,population,population,age,age,debit_card,got_savings,borrowed,given,...,Q19,Q20,Q21,Q22,Q23,Q24,Q25,Q26,Q27,target
0,0.478657,Morocco,2.836511,25550170.0,66.0,66.0,0.989621,2.079791,2.696864,0.485135,...,2.0,1.0,1,2,NaN,1,2,2.0,2.0,0
1,0.829662,Lebanon,2.730439,4591023.0,44.0,44.0,0.989621,2.408512,3.029042,0.485135,...,2.0,2.0,1,2,NaN,2,2,2.0,1.0,1
2,0.478657,Morocco,2.836511,25550170.0,29.0,29.0,1.250797,2.286197,2.890727,0.485135,...,2.0,2.0,1,1,2.0,2,2,2.0,NaN,0
3,0.871612,Canada,2.846823,30491168.0,44.0,44.0,1.250797,2.906658,3.029042,0.747652,...,NaN,NaN,1,1,2.0,1,2,NaN,NaN,1
4,0.216374,Mexico,2.909528,93002320.0,36.0,36.0,0.989621,2.079791,2.890727,0.485135,...,2.0,2.0,1,2,NaN,1,2,2.0,2.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104862,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,2.0,1,2,NaN,2,2,2.0,2.0,0
104863,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2,1,2.0,1,2,NaN,NaN,1
104864,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2,2,NaN,2,2,NaN,2.0,0
104865,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1,2,NaN,2,2,NaN,1.0,1


In [430]:
full_data = dt.loc[dt.Q1.isin([2,1])].loc[dt.Q6.isin([2,1])].\
loc[dt.Q10a.isin([2,1])].loc[dt.Q10b.isin([2,1])].\
loc[dt.Q11.isin([2,1])].loc[dt.Q12.isin([2,1])].\
loc[dt.Q13a.isin([2,1])].loc[dt.Q14.isin([2,1])].\
loc[dt.Q15.isin([2,1])].loc[dt.Q16.isin([2,1])].\
loc[dt.Q17a.isin([2,1])].loc[dt.Q17b.isin([2,1])].\
loc[dt.Q21.isin([2,1])].loc[dt.Q22.isin([2,1])].\
loc[dt.Q24.isin([2,1])].loc[dt.Q25.isin([2,1])]

In [431]:
full_data = full_data[['country', 'population', 'age', 'debit_card', 'got_savings', 'borrowed',
       'given', 'benefit','target']]

In [432]:
target = full_data.target
full_data.pop('target')

0         0
1         1
2         0
3         1
4         1
         ..
104862    0
104863    1
104864    0
104865    1
104866    0
Name: target, Length: 98141, dtype: int64

In [433]:
full_data

,country,population,age,debit_card,got_savings,borrowed,given,benefit
0,0.478657,2.836511,66.0,0.989621,2.079791,2.696864,0.485135,1.700711
1,0.829662,2.730439,44.0,0.989621,2.408512,3.029042,0.485135,1.675943
2,0.478657,2.836511,29.0,1.250797,2.286197,2.890727,0.485135,1.675943
3,0.871612,2.846823,44.0,1.250797,2.906658,3.029042,0.747652,1.925770
4,0.216374,2.909528,36.0,0.989621,2.079791,2.890727,0.485135,1.700711
...,...,...,...,...,...,...,...,...
104862,0.452849,3.028820,48.0,0.989621,2.079791,2.890727,0.485135,1.675943
104863,0.753890,2.753077,16.0,1.250797,2.614918,2.890727,0.747652,1.925770
104864,0.455587,2.627547,53.0,1.250797,2.079791,2.890727,0.747652,1.901002
104865,0.889855,2.895362,50.0,0.989621,2.573029,2.890727,0.747652,1.901002


In [451]:
full_data

,country,population,age,debit_card,got_savings,borrowed,given,benefit
0,0.478657,2.836511,66.0,0.989621,2.079791,2.696864,0.485135,1.700711
1,0.829662,2.730439,44.0,0.989621,2.408512,3.029042,0.485135,1.675943
2,0.478657,2.836511,29.0,1.250797,2.286197,2.890727,0.485135,1.675943
3,0.871612,2.846823,44.0,1.250797,2.906658,3.029042,0.747652,1.925770
4,0.216374,2.909528,36.0,0.989621,2.079791,2.890727,0.485135,1.700711
...,...,...,...,...,...,...,...,...
104862,0.452849,3.028820,48.0,0.989621,2.079791,2.890727,0.485135,1.675943
104863,0.753890,2.753077,16.0,1.250797,2.614918,2.890727,0.747652,1.925770
104864,0.455587,2.627547,53.0,1.250797,2.079791,2.890727,0.747652,1.901002
104865,0.889855,2.895362,50.0,0.989621,2.573029,2.890727,0.747652,1.901002


In [452]:
%%time
parameters = {
    'n_estimators': [5, 10, 15, 20, 25, 50, 100],
    'learning_rate': [0.01, 0.05, 0.1],
    'num_leaves': [7, 15, 31],
}

model_lgbm = lgbm.LGBMClassifier(
    random_state=42,
    device =  "gpu"
)

model_lgbm = GridSearchCV(
    model_lgbm, 
    parameters, 
    cv=5,
    scoring='roc_auc',
)

model_lgbm.fit(
    X_train, 
    y_train
)

print('-----')
print(f'Best parameters {model_lgbm.best_params_}')
print(
    f'Mean cross-validated accuracy score of the best_estimator: ' + 
    f'{model_lgbm.best_score_:.3f}'
)
cross_valid_scores['lightgbm'] = model_lgbm.best_score_
print('-----')

-----
Best parameters {'learning_rate': 0.1, 'n_estimators': 100, 'num_leaves': 31}
Mean cross-validated accuracy score of the best_estimator: 0.786
-----
CPU times: user 2min 44s, sys: 38.5 s, total: 3min 23s
Wall time: 2min 7s


In [462]:
model_lgbm = lgbm.LGBMClassifier(
    learning_rate = .01,n_estimators=100,num_leaves=31,
    random_state=42,
    device =  "gpu"
)

In [463]:
model_lgbm.fit(full_data,target)

LGBMClassifier(device='gpu', learning_rate=0.01, random_state=42)

In [455]:
def create_submission(model, test, model_name):
    y_pred_test = model.predict_proba(test.drop(['ID'],axis=1))[:, 1]
    sub = pd.DataFrame(
        {
            'ID': submission.ID, 
            'target': y_pred_test,
        }
        
    )
    sub.to_csv(f"submission_{model_name}.csv", index=False)
    
    return y_pred_test

In [465]:
submition['target'] = model_lgbm.predict_proba(full_test)[:,1]

In [466]:
submition.to_csv('last.csv',index=False)